The code in this notebook is designed to provide researchers with a visual confirmation of the OpenStreetMap data queried using the code in related notebooks #1-#4. This notebook is intended to be used with NTD data that includes a column with Facility IDs and Lon, Lat coordinates. The code can be modified to visualize station areas using data with a different file structure.

# **Visualize Station Coordinates, Circular Buffer, and OpenStreetMap Layer**

In [7]:

import time
import os
from ipywidgets import interact, widgets
from IPython.display import display as ipy_display, clear_output
import folium
from folium import CircleMarker, Circle
import pandas as pd

def plot_map(desired_facility_id, gdf):
    try:
        # Check if 'Facility ID' column exists
        if 'Facility ID' not in gdf.columns:
            print("Facility ID column not found in the data.")
            return

        # Convert the input to integer
        facility_id = int(desired_facility_id)
        filtered_gdf = gdf[gdf['Facility ID'] == facility_id]

        # Proceed only if the Facility ID exists in the data
        if filtered_gdf.empty:
            print("Facility ID not found.")
            return

        # Extract the coordinates
        coords = filtered_gdf.iloc[0]['coordinates'].strip('()').split(', ')
        lat = float(coords[0])  # Note the change in index
        lon = float(coords[1])  # Note the change in index

    except ValueError:
        print("Invalid input. Please enter a numeric Facility ID.")
        return

    # Set up the map centered around the station
    m = folium.Map(location=[lat, lon], zoom_start=16, control_scale=True)

    # Add a small red dot at the center of the circle
    CircleMarker(location=[lat, lon], radius=3, color='red', fill=True, fill_color='red').add_to(m)

    # Add a circle of radius 1/4 mile (in meters) around the station
    Circle(location=[lat, lon], radius=402.336).add_to(m)

    # Add a layer control panel to the map
    m.add_child(folium.LayerControl())

    # Explicitly display the map
    ipy_display(m)

# Read the .csv file into a DataFrame
gdf = pd.read_csv('Final__NTD_Station_Data.csv')

# Create an interactive widget for Facility ID input
facility_id_input = widgets.Text(
    value='',
    placeholder='Enter Facility ID',
    description='Facility ID:',
    disabled=False
)

# Create a button to trigger the map plotting
plot_button = widgets.Button(description="Show Map")

# Define the button click event handler
def on_button_clicked(b):
    clear_output(wait=True)
    plot_map(facility_id_input.value, gdf)

# Attach the event handler to the button
plot_button.on_click(on_button_clicked)

# Display the widgets
ipy_display(facility_id_input, plot_button)